# Explore the datasets that are not PAN 
- Find out the distribution/population of the geospatial metadata fields 
- Can we directly use GeoNames to get the entities? 
- Between step: add administrative levels 

In [1]:
import sys
# Specify the path to the scripts folder
sys.path.append('../scripts/')
from mongodb import load_data

In [2]:
collection = load_data()

In [3]:
# This query selects all records that are not PAN and have no geospatial information 
query = {
    "$and": [
        {"ore:describes.dansTemporalSpatial:dansSpatialPoint": {"$exists": False}},
        {"ore:describes.dansTemporalSpatial:dansSpatialBox": {"$exists": False}},
        # condition that the author field is not PAN: 
        {"ore:describes.author.citation:authorName": {"$ne": "Portable Antiquities of the Netherlands"}}
    ]
}

In [4]:
nopan = collection.find(query)

In [5]:
nopan.count()

/tmp/ipykernel_38918/2780690237.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nopan.count()


5878